In [27]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os
import numpy as np
from gc import callbacks
import scipy.linalg as la

In [ ]:
# read the file
# mortality
zipped = True
if zipped:
    mdf_path = r'data/mdf_final.csv'
    mdf = pd.read_csv(mdf_path, low_memory=False)
    mdf = mdf.drop(columns='Unnamed: 0')
    
    pdf_path = r'data\pdf_final.csv'
    pdf = pd.read_csv(pdf_path, low_memory=False)
    pdf = pdf.drop(columns=['Lb', 'Unnamed: 0'])
    
    lnmrdf = pd.read_csv(r'data/lnmr_final.csv', low_memory=False)
    lnmrdf = lnmrdf.drop(columns='Unnamed: 0')
    
    
else:
    mortality_path_7 = r'data/MortIcd7.csv'
    mortality_data_7 = pd.read_csv(mortality_path_7, low_memory=False)

    mortality_path_8 = r'data/MortIcd8.csv'
    mortality_data_8 = pd.read_csv(mortality_path_8, low_memory=False)

    mortality_path_9 = r'data/Morticd9.csv'
    mortality_data_9 = pd.read_csv(mortality_path_9, low_memory=False)

    mortality_path_2002 = r'data/Morticd10_part1.csv'
    mortality_data_2002 = pd.read_csv(mortality_path_2002, low_memory=False)

    mortality_path_2007 = r'data/Morticd10_part2.csv'
    mortality_data_2007 = pd.read_csv(mortality_path_2007, low_memory=False)

    mortality_path_2012 = r'data/Morticd10_part3.csv'
    mortality_data_2012 = pd.read_csv(mortality_path_2012, low_memory=False)

    mortality_path_2016 = r'data/Morticd10_part4.csv'
    mortality_data_2016 = pd.read_csv(mortality_path_2016, low_memory=False)

    mdf = pd.concat([mortality_data_7, mortality_data_8, mortality_data_9, mortality_data_2002, mortality_data_2007, mortality_data_2012, mortality_data_2016], ignore_index=True)
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)

    # population
    pop_path = r'data/pop.csv'
    pdf = pd.read_csv(pop_path, low_memory=False)
    dropped = False
    if not dropped:
        undesired_population_columns = ['Admin1', 'SubDiv', 'Sex', 'Frmat', 'Lb'] 
        pdf = pdf.drop(columns = undesired_population_columns)

# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']

In [ ]:
# needs to swap out the nah columns into averages 
done = True
if not done:
    not_used_country_list = []
    country_num_count = 0
    not_useful = False
    for key in country_code_map:
        country_num_count += 1
        display(country_num_count / len(country_code_map))
        country_mdf = mdf[mdf['Country'] == key]
        
        if country_mdf.empty:
            not_used_country_list.append(key)
        
        else:
            min_year = int(country_mdf['Year'].min())
            max_year = int(country_mdf['Year'].max()) 
            for i in range (min_year, max_year + 1):
                
                if not_useful:
                    break
                
                mask = (mdf['Country'] == key) & (mdf['Year'] == i)
                curr_Year = i

                for c in mdf.columns.values:
                    
                    if not_useful:
                        break
                    
                    average = mdf.loc[mask, c].mean()   
                    
                    max_iter = max_year - curr_Year
                    count = 0 
                    while np.isnan(average):
                        curr_Year += 1
                        count += 1
                        mask_alt = (mdf['Country'] == key) & (mdf['Year'] == curr_Year) 
                        average = mdf.loc[mask_alt, c].mean()   
                        
                        if count > max_iter:
                            not_useful = True
                            break 
                        
                        
                    mdf.loc[mask, c] = mdf.loc[mask, c].fillna(average)
                    curr_Year = i 
                    
            if not_useful:
                display(key)
                not_used_country_list.append(key)
                index = mdf[mdf['Country'] == key].index
                mdf = mdf.drop(index)
                not_useful = False
                
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)            
    mdf = mdf.groupby(['Country', 'Year'], as_index=True).agg("sum").reset_index()
    
    # building the potential country list 
    country_list = list(set(mdf['Country']))
    country_list.sort()
    len(country_list) 
        
            

        

In [ ]:
# build pdf
done = True
if not done:
    not_used_pop_list = []
    country_num_count = 0
    not_useful = False
    for key in country_list:
        
            country_num_count += 1
            display(country_num_count / len(country_list))
            
            country_pdf = pdf[pdf['Country'] == key]
            
            if country_pdf.empty:
                not_used_pop_list.append(key)
            
            else:
                min_year = int(country_pdf['Year'].min())
                max_year = int(country_pdf['Year'].max()) 
                
                if min_year == max_year:
                    if (country_pdf.isnull().sum().sum() > 0):
                        not_useful = True
                    else:
                        continue
                
                for i in range (min_year, max_year + 1):
                    
                    if not_useful:
                        break
                    
                    mask = (pdf['Country'] == key) & (pdf['Year'] == i)
                    curr_Year = i

                    for c in pdf.columns.values:
                        
                        if not_useful:
                            break
                        
                        average = pdf.loc[mask, c].mean()   
                        
                        max_iter = max_year - curr_Year
                        count = 0 
                        while np.isnan(average):
                            curr_Year += 1
                            count += 1
                            mask_alt = (pdf['Country'] == key) & (pdf['Year'] == curr_Year)
                            average = pdf.loc[mask_alt, c].mean()   
                            
                            if count > max_iter:
                                not_useful = True
                                break 
                            
                            
                        pdf.loc[mask, c] = pdf.loc[mask, c].fillna(average)
                        curr_Year = i 
                        
                if not_useful:
                    display(key)
                    not_used_pop_list.append(key)
                    index = pdf[pdf['Country'] == key].index
                    pdf = pdf.drop(index)
                    not_useful = False
                    
    # delete the unused countries 
    for key in not_used_pop_list:
        country_list.remove(key)
        
    # finalizing pdf 
    pdf = pdf[pdf['Country'].isin(country_list)]
    pdf = pdf.groupby(['Country', 'Year'], as_index=True).agg("sum").reset_index()
    pdf.to_csv(r'data/pdf_final.csv')
    
    # finalizing mdf 
    mdf = mdf[mdf['Country'].isin(country_list)]
    mdf.to_csv(r'data/mdf_final.csv')
                    
                
        
    

In [ ]:
# create the log mortality rate graph 

done = True
if not done:
    
    lnmr_dict = {}
    lnmr_dict['Country'] = []
    lnmr_dict['Year'] = []
    for i in range (6, 27):
        lnmr_column = f'Log Mortality{i}'
        lnmr_dict[lnmr_column] = []
    
    lnmrdf = pd.DataFrame(lnmr_dict)
    
    country_list = list(set(mdf['Country']))
    country_list.sort()
    
    country_num_count = 0
    for key in country_list:
        
        country_num_count += 1
        display(country_num_count / len(country_list))
        
        country_mdf = mdf[mdf['Country'] == key]
        min_year = int(country_mdf['Year'].min())
        max_year = int(country_mdf['Year'].max()) 
        
        for i in range (min_year, max_year + 1):
            
            m_mask = (mdf['Country'] == key) & (mdf['Year'] == i)
            p_mask = (pdf['Country'] == key) & (pdf['Year'] == i)
            
            if pdf[p_mask].empty or mdf[m_mask].empty:
                continue
            else:
                lnmrdf.loc[lnmrdf.shape[0]] = ['None'] * lnmrdf.shape[1]
                lnmrdf.loc[lnmrdf.shape[0]-1, ['Country']] = key
                lnmrdf.loc[lnmrdf.shape[0]-1, ['Year']] = i
            
            for c in range(6, 27):
                
                if c == 6:
                    deaths = 0
                    population = 0
                    for j in range (2, 7):
                        m_column = f'Deaths{c}'
                        p_column = f'Pop{c}'
                        lnmr_column = f'Log Mortality{c}'
                    try:
                        deaths += mdf[m_mask][m_column].values.astype('float')[0]
                        population += pdf[p_mask][p_column].values.astype('float')[0]
                    except:
                        display(key)
                        display(i)
                        display(c)
                        callbacks()
                else:
                    m_column = f'Deaths{c}'
                    p_column = f'Pop{c}'
                    lnmr_column = f'Log Mortality{c}'
                    try:
                        deaths = mdf[m_mask][m_column].values.astype('float')[0]
                        population = pdf[p_mask][p_column].values.astype('float')[0]
                    except:
                        display(key)
                        display(i)
                        display(c)
                        callbacks()
                try:
                    ratio = deaths / population
                    lnmrdf.loc[lnmrdf.shape[0]-1, lnmr_column] = np.log(ratio)
                except:
                    display(key)
                    display(i)
                    display(c)
                    callbacks()
    
    lnmrdf.to_csv(r'data/lnmr_final.csv')

In [102]:
key, i = 2140, 1956

country_lnmrdf = lnmrdf[lnmrdf['Country'] == key]
beginning_mask = (lnmrdf['Country'] == key) & (lnmrdf['Year'] >= i) & (lnmrdf['Year'] < i + 4)
ending_mask = (lnmrdf['Country'] == key) & (lnmrdf['Year'] <= i + 4) & (lnmrdf['Year'] > i)
beginning_year = np.matrix(lnmrdf[beginning_mask].drop(columns=['Country', 'Year']))
ending_year = np.matrix(lnmrdf[ending_mask].drop(columns=['Country', 'Year'])) 



matrix_of_change = np.nan_to_num(ending_year - beginning_year)

a = matrix_of_change.mean(axis=0)

demeaned_matrix = matrix_of_change - a

u, s, vh = np.linalg.svd(demeaned_matrix)

v = np.transpose(vh)
k = (u[0] * s[0])
b = v[0].reshape(21, 1)

display(k * b)

scalar = demeaned_matrix / (k*b)

display(demeaned_matrix)

display(scalar)
                

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11888\2022814032.py:11: RuntimeWarning: invalid value encountered in subtract
  matrix_of_change = np.nan_to_num(ending_year - beginning_year)


matrix([[ 8.96766035e-18, -1.42088960e-17,  2.23644585e-17,
          2.52895718e-02, -1.60726151e-02,  5.11410447e-02,
         -2.19387581e-02,  2.70248557e-02,  5.16448237e-02,
          1.31756633e-01, -5.82311227e-03,  6.96859877e-02,
         -1.03361173e-01, -1.19918825e-02,  8.23271465e-02,
         -7.70053473e-02,  4.66210978e-02,  1.09347108e-03,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-1.87479190e-17,  2.97053212e-17, -4.67554569e-17,
         -5.28707407e-02,  3.36016389e-02, -1.06916200e-01,
          4.58654815e-02, -5.64985499e-02, -1.07969407e-01,
         -2.75452301e-01,  1.21738818e-02, -1.45686523e-01,
          2.16088346e-01,  2.50704011e-02, -1.72114310e-01,
          1.60988481e-01, -9.74667341e-02, -2.28602629e-03,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-2.07049233e-17,  3.28061156e-17, -5.16360321e-17,
         -5.83896607e-02,  3.71091510e-02, -1.18076663e-01,
          5.06531565e-02, -6.23961594e

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_11888\2022814032.py:25: RuntimeWarning: invalid value encountered in divide
  scalar = demeaned_matrix / (k*b)


matrix([[ 0.        ,  0.1229243 , -0.21482393,  0.16738668, -0.00362455,
          0.06687248,  0.02214501,  0.05280219,  0.11298808,  0.19189974,
          0.09318989,  0.12525541, -0.1050478 ,  0.00409886,  0.15003924,
         -0.04671532,  0.05831998,  0.03595247,  0.        ,  0.        ,
          0.        ],
        [ 0.        , -0.07891516,  0.24848123,  0.09540605, -0.17762399,
         -0.00359405, -0.15297308, -0.08587317, -0.0984899 , -0.09409443,
         -0.21913221, -0.10323262, -0.08260632,  0.01135962, -0.12719401,
          0.03149779,  0.02779835, -0.15467511,  0.        ,  0.        ,
          0.        ],
        [ 0.        , -0.08718847, -0.22057996, -0.16869867,  0.02993687,
         -0.02453154,  0.1376615 ,  0.15366178,  0.13681996, -0.17319913,
         -0.00058706,  0.30325453,  0.17549979,  0.10030383,  0.1449666 ,
          0.18530356,  0.08450857,  0.2374609 ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.04317933,  0.18692

matrix([[ 0.00000000e+00, -8.65122082e+15, -9.60559503e+15,
          6.61880238e+00,  2.25511027e-01,  1.30760885e+00,
         -1.00940118e+00,  1.95383784e+00,  2.18779100e+00,
          1.45647122e+00, -1.60034512e+01,  1.79742610e+00,
          1.01631783e+00, -3.41802634e-01,  1.82247586e+00,
          6.06650276e-01,  1.25093540e+00,  3.28792125e+01,
                     nan,             nan,             nan],
        [-0.00000000e+00, -2.65660023e+15, -5.31448609e+15,
         -1.80451506e+00, -5.28617046e+00,  3.36155575e-02,
         -3.33525519e+00,  1.51991814e+00,  9.12201956e-01,
          3.41599725e-01, -1.80001918e+01,  7.08594153e-01,
         -3.82280287e-01,  4.53108697e-01,  7.39008888e-01,
          1.95652450e-01, -2.85208549e-01,  6.76611248e+01,
                     nan,             nan,             nan],
        [-0.00000000e+00, -2.65768948e+15,  4.27182250e+15,
          2.88918736e+00,  8.06724731e-01,  2.07759460e-01,
          2.71772802e+00, -2.46268004e

In [28]:
# fit the model 
# construct the matrix M, of log mortalit rate changes, second year to last year
# time span will be 5 years, since the age group is divided in 5 
done = False
if not done:
    country_list = list(set(lnmrdf['Country']))
    country_list.sort()
    country_mortality_model_estimate = {}
    country_count = 0
    for key in country_list:
        
        country_count += 1
        display(country_count / len(country_list))
        
        country_year_mortality_model = {}
        country_lnmrdf = lnmrdf[lnmrdf['Country'] == key]
        min_year = int(country_lnmrdf['Year'].min())
        max_year = int(country_lnmrdf['Year'].max())
        year_span = 5
        beginning_mask = (lnmrdf['Country'] == key) & (lnmrdf['Year'] == min_year)
        beginning_index = min_year
        
        for i in range (min_year + year_span, max_year, year_span):
           
                ending_mask = (lnmrdf['Country'] == key) & (lnmrdf['Year'] == i)
                
                beginning_year = lnmrdf[beginning_mask].values[0][2:]
                ending_year = lnmrdf[ending_mask].values[0][2:]
                
                matrix_of_change = np.nan_to_num(ending_year - beginning_year)
                
                a = matrix_of_change / year_span
                
                m_hat = matrix_of_change - a
                try:
                    u, singular, V_transpose = np.linalg.svd(m_hat)
                    v = np.transpose(V_transpose)
                    
                    k = u[0] * singular[0][0]
                    b = v[0]
                    
                    scalar = m_hat / (b * k)
                    
                    k *= scalar
                    b = b / scalar
                    
                    country_year_mortality_model[beginning_index] = {'a': a, 'b': b, 'k': k}
                    
                    beginning_mask = ending_mask
                    beginning_index = i
                except:
                    display(key, i)
                    callbacks()
                    
                
        country_mortality_model_estimate[key] = country_year_mortality_model   
                
                

        

0.02857142857142857

2140

1961

TypeError: 'list' object is not callable

In [ ]:
country_mortality_model_estimate

In [ ]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
undesired = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4']
modified_dict = {}
for i in mdf.columns.values:
    if i not in undesired:
       modified_dict[i] = []

modified_df = pd.DataFrame(modified_dict)
list_delete_keys = []
consider_population = True

for key in country_code_map: 
    temp_mortality_df = mdf[mdf['Country'] == key] # filter to the country
    temp_pop_data = pop_data[pop_data['Country'] == key]
    if temp_mortality_df.empty:
        list_delete_keys.append(key)
    else:
        for i in range (1980, 2003):
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i] # filter to the year 
            temp_temp_pop_data = temp_pop_data.loc[temp_pop_data['Year'] == i] 
            if temp_temp_mortality_df.empty:
                continue
            else:
                modified_df.loc[modified_df.shape[0]] = ['None'] * modified_df.shape[1]
                modified_df.loc[modified_df.shape[0]-1, ['Country']] = key
                modified_df.loc[modified_df.shape[0]-1, ['Year']] = i
                column_name = modified_df.columns.values 
                k = 1
                for c in column_name[2:]:
                    j = f'Pop{k}'
                    population = temp_temp_pop_data[j].sum()
                    if c in temp_temp_mortality_df.columns.values:
                        if consider_population:
                            if population == 0:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = 0
                            else:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() / population
                        else:    
                            modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() 
                    k += 1
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()      

In [ ]:
# building the mortality model 
country_mortality_model = {}
final_beginning_year = 9999
for key in country_code_map:
    # isolate country
    temp_modified_df = modified_df[modified_df['Country'] == key]
    # temp_modified_df.drop(columns=['Country'], inplace = True) 
    
    # finding alpha 
    last_year = temp_modified_df['Year'].max()
    
    if last_year < final_beginning_year:
        final_beginning_year = last_year
    
    first_year = temp_modified_df['Year'].min()
    duration = last_year - first_year + 1
    last_row = temp_modified_df[temp_modified_df['Year'] == last_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    first_row = temp_modified_df[temp_modified_df['Year'] == first_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    aphla = (last_row - first_row) / duration

    country_mortality_model[key] = {'a': aphla, 'd': duration}

In [ ]:
final_ending_year = 2050
current_year = 2000

mortality_pred_all = []
while current_year <= final_ending_year:
    
    mortality_pred_year = {}
    
    for key in country_code_map: 
        
        print(key)
        
        # isolate the data to the target data row \n",
        temp_modified_df = modified_df[(modified_df['Country'] == key)]  
        # check if there is real data for this year for this country\n",
        if current_year <= 2002:
            df = temp_modified_df[temp_modified_df['Year'] == current_year + 1]
            if not df.empty:
                df = df.loc[:, ~df.columns.isin(['Country', 'Year'])].values.astype('float')
                mortality_pred_year[key] = df 
                continue
 
        # clean the data for the row
        temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
        # find the aphla, b and k value for the model for future \n",
        aphla = country_mortality_model[key]['a']
        duration = country_mortality_model[key]['d']
        m_hat = (temp_modified_df - aphla)
        s, u, v = np.linalg.svd(m_hat)
        scalar = s[0]
        k = scalar * u[0]
        b = v[0] / scalar 
  
        # compute the next years result",
        change = np.exp(aphla + b * k)
        display(temp_modified_df.shape)
        display(change.shape)
        next_year = temp_mortality_df + change
        mortality_pred_year[key] = next_year
        
        # update parameters
        duration += 1
        aphla = (aphla * duration + change) / duration
        country_mortality_model[key]['a'] = aphla
        country_mortality_model[key]['d'] = duration
    
    current_year += 1
    mortality_pred_all.append(mortality_pred_year)
        
        


In [ ]:
key = 1125
temp_modified_df = modified_df[(modified_df['Country'] == key)]  
# clean the data for the row
temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
# find the aphla, b and k value for the model for future \n",
aphla = country_mortality_model[key]['a']
duration = country_mortality_model[key]['d']
m_hat = (temp_modified_df - aphla)
s, u, v = np.linalg.svd(m_hat)
scalar = s[0]
k = scalar * u[0]
b = v[0] / scalar 

display(aphla)
display(k, b)
change = np.exp(aphla + b * k)
display(temp_modified_df)
display(change)
display(temp_modified_df.shape)
next_year = temp_mortality_df + change
mortality_pred_year[key] = next_year